In [27]:
from numba import cuda
major, minor = cuda.get_current_device().compute_capability
print(f'GPU compute capability: {major}.{minor}')

GPU compute capability: 7.5


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [11]:
dir = "/content/drive/MyDrive/Colab Notebooks/Parallel Programming/Project/"
%cd $dir

/content/drive/MyDrive/Colab Notebooks/Parallel Programming/Project


In [4]:
!nvidia-smi


Tue Dec 10 07:39:29 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   41C    P8               9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [31]:
!apt-get update
!apt-get install -y cuda

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:3 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:4 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:5 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:7 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Reading package lists... Done
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
Reading package lists... Done
Building dependency tree... Done
Reading

In [6]:
!apt-get install -y cmake

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
cmake is already the newest version (3.22.1-1ubuntu1.22.04.2).
0 upgraded, 0 newly installed, 0 to remove and 55 not upgraded.


In [106]:
with open(dir + 'CMakeLists.txt', 'w') as f:
    f.write("""
cmake_minimum_required(VERSION 3.10)
project(MyCudaProject)

# Find CUDA package
find_package(CUDA REQUIRED)

# Set CUDA flags (optional optimization flags)
set(CUDA_NVCC_FLAGS "-O3 -gencode arch=compute_75,code=sm_75")  # Set sm_75 for Tesla T4 or similar GPUs

# Include CUDA directories
include_directories(${CUDA_INCLUDE_DIRS})

# Add the source files and handle CUDA source files with nvcc
cuda_add_executable(Project src/main.cc src/kernel.cu src/custom/gpu-support.cu)

# Link CUDA libraries
target_link_libraries(Project ${CUDA_LIBRARIES})
""")

In [107]:
# Navigate to your build directory
%cd $dir/build

/content/drive/MyDrive/Colab Notebooks/Parallel Programming/Project/build


In [115]:
!make clean

In [116]:
!cmake ..

CMake Warning (dev) at CMakeLists.txt:6 (find_package):
  Policy CMP0146 is not set: The FindCUDA module is removed.  Run "cmake
  --help-policy CMP0146" for policy details.  Use the cmake_policy command to
  set the policy and suppress this warning.

This warning is for project developers.  Use -Wno-dev to suppress it.

-- Configuring done (0.1s)
-- Generating done (0.3s)
-- Build files have been written to: /content/drive/MyDrive/Colab Notebooks/Parallel Programming/Project/build


In [117]:
!make

[-25%] Building NVCC (Device) object CMakeFiles/Project.dir/src/custom/Project_generated_gpu-support.cu.o
[  0%] Building NVCC (Device) object CMakeFiles/Project.dir/src/Project_generated_kernel.cu.o
[ 25%] Building CXX object CMakeFiles/Project.dir/src/main.cc.o
[ 50%] Linking CXX executable Project
[ 50%] Built target Project


In [118]:
# Run the compiled CUDA executable
!./Project

**********GPU info**********
Name: Tesla T4
Compute capability: 7.5
Num SMs: 40
Max num threads per SM: 1024
Max num warps per SM: 32
GMEM: 15835660288 bytes
CMEM: 65536 bytes
L2 cache: 4194304 bytes
SMEM / one SM: 65536 bytes
****************************
Hello from Host!
Hello from Device!
